In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Optical-potential model (onda S) para fusión ligera

Calcula, en el rango 0–1 MeV, 
  • parte real e imaginaria de δ0(E)
  • σ(E)  (sección eficaz)
  • S(E)  (factor astrofísico)
  • u0(r)  y  du0/dr  para un valor de E
graficando cada magnitud.

Autor:  (tu nombre)   2025
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import physical_constants, pi
from scipy.special import coulombf, coulombg

# =====================  CONSTANTES BÁSICAS  =============================== #
ħc      = 197.3269804          # [MeV·fm]
α_fine  = 1/137.035999084
e2_4pie0= α_fine * ħc          # e²/(4πϵ0)  [MeV·fm]
m_u     = physical_constants['atomic mass constant energy equivalent in MeV'][0]

# =====================  FUNCIONES AUXILIARES  ============================= #
def reduced_mass(Ap, At):
    """Masa reducida en MeV/c² (A en unidades de masa atómica)."""
    return Ap * m_u * At / (Ap + At)

def coulomb_FG(l, η, ρ):
    """Devuelve F_l, G_l y sus derivadas (l=0)."""
    F   = coulombf(l, η, ρ)
    G   = coulombg(l, η, ρ)
    # derivada numérica muy precisa (ρ~fm, paso 1e-5 fm)
    h   = 1e-5
    Fp  = (coulombf(l, η, ρ+h) - coulombf(l, η, ρ-h)) / (2*h)
    Gp  = (coulombg(l, η, ρ+h) - coulombg(l, η, ρ-h)) / (2*h)
    return F, G, Fp, Gp

def delta0(E, Zp, Zt, Ap, At, Vr, Vi, r0):
    """Devuelve δ0(E) complejo (rad)."""
    mr   = reduced_mass(Ap, At)               # MeV
    k    = np.sqrt(2*mr*E)/ħc                # fm⁻¹
    ac   = ħc**2 / (mr*e2_4pie0*Zp*Zt)       # fm
    η    = 1/(k*ac)
    RN   = r0 * (At**(1/3) + Ap**(1/3))      # fm
    # número de onda dentro del pozo (complejo)
    kN   = np.sqrt(2*mr*(E-Vr-1j*Vi))/ħc
    ρ    = k * RN
    F, G, Fp, Gp = coulomb_FG(0, η, ρ)

    # Expresiones abreviadas
    cot_kN_RN = (np.cos(kN*RN)/np.sin(kN*RN))
    num   = kN*cot_kN_RN*G - k*Gp
    den   = k*Fp - kN*cot_kN_RN*F
    W     = num/den                             # W_r + i W_i  = cot δ0
    δ0    = np.arctan(1/W)                      # atan devuelve rama correcta
    return δ0, W, k, ac

def sigma_S(E, k, W, mr, Zp, Zt):
    """σ(E) y S(E) (E en MeV, k en fm⁻¹)."""
    Wr, Wi = W.real, W.imag
    σ   = pi/k**2 * (-4*Wi)/(Wr**2 + (Wi-1)**2)  # barn si k en fm⁻¹
    BG  = Zp*Zt*e2_4pie0*np.sqrt(2*mr)/ħc        # √(MeV)·fm⁰ → constante BG
    S   = σ * E * np.exp(BG/np.sqrt(E))
    return σ, S

def wavefunction(r, E, params):
    """Devuelve u0(r,E) y su derivada, normalizando B=1."""
    (Zp,Zt,Ap,At,Vr,Vi,r0) = params
    mr = reduced_mass(Ap, At)
    k  = np.sqrt(2*mr*E)/ħc
    RN = r0 * (At**(1/3)+Ap**(1/3))
    δ0, W, k_val, ac = delta0(E,*params[:4],Vr,Vi,r0)[:4]
    kN= np.sqrt(2*mr*(E-Vr-1j*Vi))/ħc
    # coeficientes (B=1) y D ajustado por continuidad en r=RN
    B   = 1.0
    F0, G0, Fp0, Gp0 = coulomb_FG(0, 1/(k_val*ac), k_val*RN)
    cotδ = 1/np.tan(δ0)
    D   = (B*np.sin(kN*RN)) / (F0*cotδ + G0)
    # regiones
    u   = np.where(r<RN,
                   B*np.sin(kN*r),
                   D*(F0*cotδ + G0)   # se ajustará punto a punto justo abajo
                   )
    # derivada por trozos
    du = np.where(r<RN,
                  B*kN*np.cos(kN*r),
                  D*(Fp0*cotδ + Gp0)*k_val)
    return u, du

# =====================  ENTRADA DE DATOS  ================================ #
print("Introduzca datos del sistema (carga Z, masa A en uma)")
Zp = int(input(" Z del proyectil = "))
Zt = int(input(" Z del blanco    = "))
Ap = float(input(" A proyectil     = "))
At = float(input(" A blanco        = "))
Vr = float(input(" Parte real Vr   [MeV] = "))
Vi = float(input(" Parte imag Vi   [MeV] = "))
r0 = float(input(" r0 (fm)         = "))

params = (Zp, Zt, Ap, At, Vr, Vi, r0)

# =====================  CÁLCULOS ENERGÉTICOS ============================= #
E_MeV = np.linspace(1e-3, 1.0, 400)          # 0 – 1 MeV
δ0_cplx = []
σ_list  = []
S_list  = []
for E in E_MeV:
    δ0, W, k, _ = delta0(E, *params)
    σ, S  = sigma_S(E, k, W, reduced_mass(Ap,At), Zp, Zt)
    δ0_cplx.append(δ0)
    σ_list.append(σ)
    S_list.append(S)

δ0_cplx = np.array(δ0_cplx)
σ_list  = np.array(σ_list)
S_list  = np.array(S_list)

# =====================  PLOTS ENERGÍA ==================================== #
fig,axs = plt.subplots(2,2,figsize=(10,8))
axs[0,0].plot(E_MeV, δ0_cplx.real*180/pi)
axs[0,0].set_xlabel('E (MeV)'); axs[0,0].set_ylabel('Re δ0 (°)')

axs[0,1].plot(E_MeV, δ0_cplx.imag*180/pi)
axs[0,1].set_xlabel('E (MeV)'); axs[0,1].set_ylabel('Im δ0 (°)')

axs[1,0].plot(E_MeV, σ_list)
axs[1,0].set_xlabel('E (MeV)'); axs[1,0].set_ylabel('σ (barn)'); axs[1,0].set_yscale('log')

axs[1,1].plot(E_MeV, S_list)
axs[1,1].set_xlabel('E (MeV)'); axs[1,1].set_ylabel('S (MeV·barn)')

fig.tight_layout()
plt.show()

# =====================  ONDA U0(r) PARA ENERGÍA DE DEMO  ================= #
E_demo = float(input("E (MeV) para graficar u0(r) = "))
r_max  = 2 * ħc**2 / (reduced_mass(Ap,At)*e2_4pie0*Zp*Zt)   # 2·ac
r      = np.linspace(1e-4, r_max, 2000)
u, du  = wavefunction(r, E_demo, params)

plt.figure(figsize=(7,4))
plt.plot(r, u.real, label='Re u0')
plt.plot(r, u.imag, label='Im u0', ls='--')
plt.xlabel('r (fm)'); plt.ylabel('u0')
plt.legend(); plt.tight_layout(); plt.show()

plt.figure(figsize=(7,4))
plt.plot(r, du.real, label='Re du0/dr')
plt.plot(r, du.imag, label='Im du0/dr', ls='--')
plt.xlabel('r (fm)'); plt.ylabel("du0/dr")
plt.legend(); plt.tight_layout(); plt.show()


ImportError: cannot import name 'coulombf' from 'scipy.special' (/home/saescobarc/venv/lib/python3.10/site-packages/scipy/special/__init__.py)

In [ ]:
pip install mpmath


In [ ]:
pip install --upgrade "scipy>=1.12"

In [2]:
python - <<'PY'
import scipy, sys
print("SciPy version:", scipy.__version__)
PY


SyntaxError: invalid syntax (143515095.py, line 1)

In [3]:
import scipy
print(scipy.__version__)


1.15.3
